# Lesson 6: Planning and Stock Report Generation

## Setup

In [27]:
llm_config={"model": "gpt-4o"}

## The task!

In [28]:
task = "Write a blogpost about the stock price performance of "\
"Nvidia in the past month. Today's date is 2024-07-11."

## Build a group chat

This group chat will include these agents:

1. **User_proxy** or **Admin**: to allow the user to comment on the report and ask the writer to refine it.
2. **Planner**: to determine relevant information needed to complete the task.
3. **Engineer**: to write code using the defined plan by the planner.
4. **Executor**: to execute the code written by the engineer.
5. **Writer**: to write the report.

In [29]:
import autogen

In [30]:
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Give the task, and send "
    "instructions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS", # always ask human to input. if not system message takes over and it refines blog post. 
)

[autogen.oai.client: 07-13 14:18:37] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [31]:
# system message is the instruction is what this agent needs to nkow. 
# description is for other agents to know - example manager knows what this agents can do. 
planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine "
    "what information is needed to complete the task. "
    "Please note that the information will all be retrieved using"
    " Python code. Please only suggest information that can be "
    "retrieved using Python code. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps. If a step fails, try to "
    "workaround",
    description="Planner. Given a task, determine what "
    "information is needed to complete the task. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps",
    llm_config=llm_config,
)

[autogen.oai.client: 07-13 14:18:37] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [32]:
engineer = autogen.AssistantAgent(
    name="Engineer",  # engineer has a default system message, no need to provide another one. 
    llm_config=llm_config,
    description="An engineer that writes code based on the plan "
    "provided by the planner.",
)

[autogen.oai.client: 07-13 14:18:37] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


**Note**: In this lesson, you'll use an alternative method of code execution by providing a dict config. However, you can always use the LocalCommandLineCodeExecutor if you prefer. For more details about code_execution_config, check this: https://microsoft.github.io/autogen/docs/reference/agentchat/conversable_agent/#__init__

In [33]:
# after engineer writes code, we need to execute this code
executor = autogen.ConversableAgent(
    name="Executor",
    system_message="Execute the code written by the "
    "engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,  # looks back 3 messages and finds the first code to execute
        "work_dir": "coding",
        "use_docker": False,
    },
)

In [34]:
writer = autogen.ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Writer."
    "Please write blogs in markdown format (with relevant titles)"
    " and put the content in pseudo ```md``` code block. "
    "You take feedback from the admin and refine your blog.",
    description="Writer."
    "Write blogs based on the code execution results and take "
    "feedback from the admin to refine the blog."
)

[autogen.oai.client: 07-13 14:18:37] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


## Define the group chat

In [35]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],  # agents in the group chat
    messages=[],
    max_round=10,
)

In [36]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)


[autogen.oai.client: 07-13 14:18:37] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


## Start the group chat!

<p style="background-color:#ECECEC; padding:15px; "> <b>Note:</b> In this lesson, you will use GPT 4 for better results. Please note that the lesson has a quota limit. If you want to explore the code in this lesson further, we recommend trying it locally with your own API key.

In [25]:
!pwd

/Users/acrobat/Documents/GitHub/autogen/DeepLearning_AI_Agentic_Design_Patterns_with_AutoGen


In [37]:
groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,  # initial task to start the conversation - write a blog post price performance....
)

Admin (to chat_manager):

Write a blogpost about the stock price performance of Nvidia in the past month. Today's date is 2024-07-11.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

To write a blog post about the stock price performance of Nvidia in the past month, you'll need to gather and analyze specific data using Python:

### Step 1: Retrieve Nvidia's Stock Data
You need historical stock data for Nvidia (NVDA) from the past month. You can use financial APIs or libraries such as `yfinance` to fetch this data.

#### Information Needed:
- Stock prices (open, high, low, close)
- Volume of stocks traded
- Date range: 2024-06-11 to 2024-07-10

### Step 2: Analyze Stock Performance
Analyze the data to get insights into Nvidia's stock performance.

#### Information Needed:
- Daily closing prices
- Percentage change in stock price
- Highest and lowest closing prices in the past month
- Average closi

## Add a speaker selection policy

In [43]:
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Give the task, and send "
    "instructions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine "
    "what information is needed to complete the task. "
    "Please note that the information will all be retrieved using"
    " Python code. Please only suggest information that can be "
    "retrieved using Python code. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps. If a step fails, try to "
    "workaround",
    description="Given a task, determine what "
    "information is needed to complete the task. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps",
    llm_config=llm_config,
)

engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="Write code based on the plan "
    "do not save or retrieve dataset from files. Keep everything in memory"
    "provided by the planner.",
)

writer = autogen.ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Writer. "
    "Please write blogs in markdown format (with relevant titles)"
    " and put the content in pseudo ```md``` code block. "
    "You take feedback from the admin and refine your blog.",
    description="After all the info is available, "
    "write blogs based on the code execution results and take "
    "feedback from the admin to refine the blog. ",
)

executor = autogen.ConversableAgent(
    name="Executor",
    description="Execute the code written by the "
    "engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
)

[autogen.oai.client: 07-13 14:35:16] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 07-13 14:35:16] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 07-13 14:35:16] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 07-13 14:35:16] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [44]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=20,
    allowed_or_disallowed_speaker_transitions={
        user_proxy: [engineer, writer, executor, planner],
        engineer: [user_proxy, executor],
        writer: [user_proxy, planner],
        executor: [user_proxy, engineer, planner],  # gives chance to planner to check the code before sending it to writer
        planner: [user_proxy, engineer, writer],
    },
    speaker_transitions_type="allowed",
)

In [45]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)

groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

[autogen.oai.client: 07-13 14:35:23] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
Admin (to chat_manager):

Write a blogpost about the stock price performance of Nvidia in the past month. Today's date is 2024-07-11.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

To write a blog post about the stock price performance of Nvidia in the past month, you'll need to gather and analyze specific data using Python:

### Step 1: Retrieve Nvidia's Stock Data
You need historical stock data for Nvidia (NVDA) from the past month. You can use financial APIs or libraries such as `yfinance` to fetch this data.

#### Information Needed:
- Stock prices (open, high, low, close)
- Volume of stocks traded
- Date range: 2024-06-11 to 2024-07-10

### Step 2: Analyze Stock Performance
Analyze the data to get insights into Nvidia's stock performance.

###

**Note**: You might experience slightly different interactions between the agents. The engineer agent may write incorrect code, which the executor agent will report and send back for correction. This process could go through multiple rounds.